In [1]:
from pathlib import Path
from polar import data_loader

ROOT_DIR = Path("..").resolve()
DATA_DIR = ROOT_DIR / "data"

In [2]:
data = data_loader.load_year_data(DATA_DIR, 2025)

In [3]:
data = data_loader.load_all_years_data(DATA_DIR)

In [4]:
def duration_to_minutes(duration_str):
    # Format: HH:MM:SS
    h, m, s = map(int, duration_str.split(":"))
    return h * 60 + m + s / 60


def safe_float(val):
    try:
        return float(val)
    except (ValueError, TypeError):
        return None


calories_stats = []
for year, months in data.items():
    for month, workouts in months.items():
        for workout in workouts:
            meta = workout["metadata"]
            sport = meta.get("Sport", "").upper()
            if sport in ["OTHER_INDOOR", "STRENGTH_TRAINING"]:
                duration = meta.get("Duration")
                calories = meta.get("Calories")
                duration_min = duration_to_minutes(duration) if duration else None
                calories_val = safe_float(calories) if calories else None
                if duration_min and calories_val:
                    cal_per_min = calories_val / duration_min
                    cal_per_hour = cal_per_min * 60
                    calories_stats.append(
                        {
                            "year": year,
                            "month": month,
                            "duration_min": duration_min,
                            "calories": calories_val,
                            "calories_per_min": cal_per_min,
                            "calories_per_hour": cal_per_hour,
                        }
                    )
calories_stats

[{'year': '2022',
  'month': '09',
  'duration_min': 44.38333333333333,
  'calories': 307.0,
  'calories_per_min': 6.917010889973714,
  'calories_per_hour': 415.02065339842284},
 {'year': '2022',
  'month': '09',
  'duration_min': 54.266666666666666,
  'calories': 477.0,
  'calories_per_min': 8.78992628992629,
  'calories_per_hour': 527.3955773955774},
 {'year': '2022',
  'month': '09',
  'duration_min': 43.25,
  'calories': 356.0,
  'calories_per_min': 8.23121387283237,
  'calories_per_hour': 493.8728323699422},
 {'year': '2022',
  'month': '09',
  'duration_min': 39.583333333333336,
  'calories': 388.0,
  'calories_per_min': 9.802105263157895,
  'calories_per_hour': 588.1263157894737},
 {'year': '2022',
  'month': '09',
  'duration_min': 17.3,
  'calories': 149.0,
  'calories_per_min': 8.61271676300578,
  'calories_per_hour': 516.7630057803468},
 {'year': '2022',
  'month': '10',
  'duration_min': 61.1,
  'calories': 510.0,
  'calories_per_min': 8.34697217675941,
  'calories_per_hour

In [5]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame(calories_stats)
fig = px.scatter(
    df,
    x="duration_min",
    y="calories_per_min",
    color="month",
    hover_data=["year", "calories", "calories_per_hour"],
    title="Calories per Minute vs Duration",
)
fig.show()

In [7]:
import plotly.express as px

df = pd.DataFrame(calories_stats)
monthly_avg = df.groupby(["year", "month"])["calories_per_hour"].mean().reset_index()

# Ensure 'year' and 'month' are strings and pad month with zeros
monthly_avg["year"] = monthly_avg["year"].astype(str)
monthly_avg["month"] = monthly_avg["month"].astype(str).str.zfill(2)
monthly_avg["date"] = monthly_avg["year"] + "-" + monthly_avg["month"]

fig = px.line(
    monthly_avg,
    x="date",
    y="calories_per_hour",
    color="year",
    markers=True,
    title="Monthly Average Calories per Hour (Timeseries)",
    labels={"date": "Date", "calories_per_hour": "Avg Calories per Hour"},
)
# fig.update_xaxes(type="category")
fig.show()